In [ ]:
import pandas as pd
import numpy as np

from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "/home/lyz/hf-models/Qwen/Qwen1.5-1.8B-Chat/",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("/home/lyz/hf-models/Qwen/Qwen1.5-0.5B-Chat/")

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
test_answer = []
for row in test.iterrows():
    content  = ''.join(open('tables/' + row[1].table).readlines()[:50])
    prompt_template = f'请结合给定的资料客观回答问题，如果问题无法从表中回答则回复无法回答，回答结果要精简，简短直接回答结果，不要分析如何回答的，直接回答，出了答案之外不需要有其他内容。\n -资料\n {content} -提问 \n {row[1].question}'

    messages = [
        {"role": "user", "content": prompt_template}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]